In [19]:
from shutil import copyfile
from pathlib import Path
import pdb

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from ipywidgets import interact
from PIL import Image
import pytesseract

In [2]:
DATA_DIR = Path('.') / 'data'

TRN_DIR = DATA_DIR / 'train'
VAL_DIR = DATA_DIR / 'valid'
MODEL_DIR = DATA_DIR / 'models'
HAND_DIR = DATA_DIR / 'handwritten'
TYPE_DIR = DATA_DIR / 'typewritten'

VAL_CSV = DATA_DIR / 'valid.csv'
TRN_CSV = DATA_DIR / 'train.csv'

BOX = 'box'
CAT = 'category'
ORI = 'original'
PATH = 'path'
CLASS = 'class'
SUB_ID = 'subject_id'
PRED_BOX = 'predicted_box'
PRED_CAT = 'predicted_category'
PRED_CLASS = 'predicted_class'
PRED_TEXT = 'predicted_text'

HANDWRITTEN = 'handwritten'
TYPEWRITTEN = 'typewritten'
CATS = ['background', HANDWRITTEN, TYPEWRITTEN]
CLASSES = len(CATS)

In [3]:
# CSV = TRN_CSV
CSV = VAL_CSV

In [4]:
df = pd.read_csv(CSV)
df[PRED_TEXT] = None

In [69]:
CONFIG = ' '.join([
    '-l eng',
    "-c tessedit_char_blacklist='€«¢»£®'",
    '',
])


def show_results(idx):
    row = df.loc[idx]
    subject_id = int(row.at[SUB_ID])
    print(subject_id)
    if row.at[PRED_CAT] != TYPEWRITTEN:
        print('Not typewritten')
        return
    path = Path(TYPE_DIR / f'{subject_id}.jpg')
    path = str(path)
    image = Image.open(path)
    text = pytesseract.image_to_string(image, config=CONFIG)

    fig, ax = plt.subplots(1, 2, figsize=(20, 10))
    plt.axis('off')

    ax[0].set_axis_off()
    ax[0].imshow(image)

    ax[1].text(0, 0.5, text, fontsize=16, verticalalignment='center')

    plt.show()


interact(show_results, idx=(0, df.shape[0] - 1));

interactive(children=(IntSlider(value=607, description='idx', max=1215), Output()), _dom_classes=('widget-inte…

In [27]:
df.to_csv(CSV, index=False)

In [ ]:
# out_dir = DATA_DIR / 'typewritten_labels'
# out_dir.mkdir()

# for idx, row in tqdm(val_df.iterrows()):
#     if row.at[PRED_CAT] != TYPEWRITTEN:
#         continue
#     src = TYPE_DIR / '{}.jpg'.format(row.at[SUB_ID])
#     dst = out_dir / '{}.jpg'.format(row.at[SUB_ID])
#     copyfile(src, dst)